# Bike-Share Demand Forecasting 1: Data Preparation

In this timeseries forecasting example we investigate demand in the [Capital Bikeshare scheme in 2011-12](https://www.capitalbikeshare.com/system-data), with relation to weather data.

This notebook downloads the [original weather-annotated dataset](http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset) from the UC Irvine website, and performs some basic preparations before uploading to S3.

Later notebooks in this series fit models to the uploaded prepared data, and compare their accuracy.

## Dependencies and configuration<a class="anchor" id="setup"/>

As usual we start by loading libraries, defining configuration, and connecting to AWS SDKs

In [ ]:
%load_ext autoreload
%autoreload 1

# External Dependencies:
import boto3
import matplotlib.pyplot as plt
import pandas as pd

# Local Dependencies:
%aimport util

In [ ]:
bucket = # TODO: Choose a bucket you've created, and SageMaker has full access to
%store bucket

# Data files to be stored in S3:
data_prefix = "data/"
%store data_prefix
target_train_filename = "target_train.csv"
%store target_train_filename
target_test_filename = "target_test.csv"
%store target_test_filename
related_filename = "related.csv"
%store related_filename

Now we connect to our AWS SDKs, and initialise our access role (which may wait a little while to ensure any newly created permissions propagate):

In [ ]:
session = boto3.Session()
region = session.region_name
s3 = session.client(service_name="s3")

## Step 1: Fetch the source data<a class="anchor" id="fetch"/>

Since this data set is comparatively small, we can process it here in the notebook instance without requiring large disk allocations.

In [ ]:
!wget -O data.zip http://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip
!rm -rf ./data/raw
!mkdir -p ./data/raw
!unzip data.zip -d ./data/raw

The [UCI dataset documentation](http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset) tells us what to expect in terms of column headers, ranges and types, which we can load here:

In [ ]:
raw_df = pd.read_csv(
    "./data/raw/hour.csv",
    index_col="instant",
    dtype={
        "dteday": str,
        "season": int,
        "yr": int,
        "mnth": int,
        "hr": int,
        "holiday": bool,
        "weekday": int,
        "workingday": bool,
        "weathersit": int,
        "temp": float,
        "atemp": float,
        "hum": float,
        "windspeed": float,
        "casual": int,
        "registered": int,
        "cnt": int
    }
).sort_values(["dteday", "hr"])
raw_df.head()

...but we also have a quick check of the ranges and variation to check our expectations:

In [ ]:
raw_df.describe()

## Step 2: Load data & interpolate gaps<a class="anchor" id="convert"/>

The raw dataset has:

* Some date/time column redundancy which we'll cut down
* No rows present for hours where total demand was zero (see above summary)

...which we'll deal with here, starting with the timestamp tidy-up:

In [ ]:
hourly_df = raw_df.copy()

# Combine day and hour into datetime column, and drop superfluous date features:
hourly_df["dteday"] = pd.to_datetime(raw_df["dteday"].map(str) + " " + raw_df["hr"].map(str) + ":00")
hourly_df = hourly_df.rename(columns={ "dteday": "timestamp" }).drop(columns=["yr", "mnth", "hr"])

Although there are missing rows for hours where nobody used bikes, there's at least one row for every day in the covered range.

Therefore we can fill in gaps pretty reliably as follows, for the benefit of any forecast methods that can't handle missing data:

1. Rider counts (casual, registered, cnt) are set to 0 (cnt=0 can be used to recover to which rows imputation was applied)
2. Daily values (season, holiday, weekday, workingday) are the mean of source records from that day (not that the aggregation should matter, because they sohuld all match)
3. Weather data (weathersit, temp, atemp, hum, windspeed) are interpolated over time from nearest present records.

Of these the weather data is the only real imputation, and time-wise interpolation should be a pretty solid approach to fill in the occasional missing hour of hourly weather data.

In [ ]:
# Compare the full range of hours covered by the data-set, to the number of records:
daterange = pd.date_range(
    start=list(hourly_df["timestamp"])[0],
    end=list(hourly_df["timestamp"])[-1],
    freq='H'
)

n_raw_records = len(hourly_df)
n_range_hours = len(daterange)
print(f"{n_raw_records} raw records vs {n_range_hours} hours in date range")
if (n_raw_records == n_range_hours):
    print("Data fully specified")
elif (n_raw_records < n_range_hours):
    # (We expect to see this)
    print("MISMATCH: Missing data will be interpolated")
elif (n_raw_records > n_range_hours):
    raise NotImplementedError("This script can't deal with duplicates yet!")

# Construct a fully range-covering table
# (including day-granularity fields taken from aggregating the source table)
tmp = pd.DataFrame({ "timestamp": daterange })
tmp["dteday"] = tmp["timestamp"].dt.strftime("%Y-%m-%d")
fill_df = tmp.merge(
    raw_df.groupby("dteday").agg("mean")[["season", "holiday", "weekday", "workingday"]],
    how="left",
    on="dteday"
).drop(columns=["dteday"])

# Join the whole-range table to our target, and fill in the day-granularity fields
assert (
    hourly_df.isna().sum().sum() == 0
), "These imputations assume no missing values in source data set records!"
imputed_df = fill_df.merge(hourly_df, how="left", on="timestamp", suffixes=("_day", ""))
imputed_df["season"].fillna(imputed_df["season_day"], inplace=True)
imputed_df["holiday"].fillna(imputed_df["holiday_day"], inplace=True)
imputed_df["weekday"].fillna(imputed_df["weekday_day"], inplace=True)
imputed_df["workingday"].fillna(imputed_df["workingday_day"], inplace=True)
imputed_df.drop(columns=["season_day", "holiday_day", "weekday_day", "workingday_day"], inplace=True)

# Fill all missing demand values with zero (which is why the records were missing in the first place)
imputed_df["casual"].fillna(0, inplace=True)
imputed_df["registered"].fillna(0, inplace=True)
imputed_df["cnt"].fillna(0, inplace=True)

# Interpolate over time for missing weather data fields:
imputed_df = imputed_df.set_index("timestamp")
imputed_df["weathersit"] = imputed_df["weathersit"].interpolate(method="time").round()
imputed_df["temp"].interpolate(method="time", inplace=True)
imputed_df["atemp"].interpolate(method="time", inplace=True)
imputed_df["hum"].interpolate(method="time", inplace=True)
imputed_df["windspeed"].interpolate(method="time", inplace=True)
imputed_df = imputed_df.reset_index()

assert not imputed_df.isna().any().any(), "Imputed DF should not have any remaining nulls!"
assert len(imputed_df) == n_range_hours, "Imputed DF should fully cover time range!"

print("Imputation complete")
imputed_df.head()

In [ ]:
# Feel free to extend feature engineering here, just leaving timestamp and *_demand columns alone:
target_suffix = "_demand"
full_df = imputed_df.rename(columns={
    "casual": f"casual{target_suffix}",
    "registered": f"registered{target_suffix}",
    "cnt": f"total{target_suffix}"
})

full_df.head()

## Step 3: Explore the data<a class="anchor" id="explore"/>

Some basic visualisations will be helpful to understand the overall structure of the timeseries.

First we will plot the overall demand across the lifetime of the full data set:

In [ ]:
fig = plt.figure(figsize=(15, 8))
ax = plt.gca()
full_df.plot(x="timestamp", y="registered_demand", ax=ax, label="Registered Customers")
full_df.plot(x="timestamp", y="casual_demand", ax=ax, label="Casual Customers")
ax.set_xlabel("Date")
ax.set_ylabel("Number of Trips")
ax.set_title("Comparison of Registered and Casual Demand - Whole Data Set")
plt.show()

In general we see from the above that:

1. "registered" or non-casual customers source the majority of demand
2. Although summer/winter seasonality can be seen in the data, the overall growth / popularity increase of the service between the two years is also significant
3. There's strong periodicity/"spikiness" at at shorter time periods than can be seen on the overall data view

**Note: Point 2 is challenging for the accuracy of our forecasts because:**

* We only have 2 years of data, so if annual seasonality looks significant we're in trouble because in general we'd like to have history covering **~5x the longest cycle time** for significant patterns
* The overall popularity of the service looks significantly **non-stationary**, which undermines accuracy for many forecasting methods as discussed in detail [here](https://cs.nyu.edu/~mohri/talks/NIPSTutorial2016.pdf)

Next we zoom in on a smaller section of the data to get a better feel for the day-to-day profile:

In [ ]:
period_df = full_df[(full_df["timestamp"] >= "2012-08-01") & (full_df["timestamp"] < "2012-09-01")]
fig = plt.figure(figsize=(15, 8))
ax = plt.gca()
period_df.plot(x="timestamp", y="registered_demand", ax=ax, label="Registered Customers")
period_df.plot(x="timestamp", y="casual_demand", ax=ax, label="Casual Customers")
ax.set_xlabel("Date")
ax.set_ylabel("Number of Trips")
ax.set_title("Comparison of Registered and Casual Demand - August 2012")
plt.show()

Here we see clearly the impact of day-of-week and time-of-day on demand, and note in particular that (in this summer month at least) casual riders make up a much higher proportion of trips at the weekends than in the week.

At these shorter timescales, our data looks a bit more stationary and like we could extract some more robust forecasts.

## Step 4: Train/test split for eodel evaluation<a class="anchor" id="split"/>

Since a lot of feature engineering has been done for us already, the main outstanding question is how we'll evaluate the accuracy of our models.

### How to evaluate forecast models

The key difference between evaluating forecasting algorithms and standard ML applications is **causality**: We must fit our model based on past data; evaluate it based on unseen "future" data; and make sure no future data is visible to the model fitting process.

**[Backtesting](https://en.wikipedia.org/wiki/Backtesting)** (or "hindcasting") is the process by which we'll perform this causal evaluation: Picking one (or maybe more) historical time points to use as a cutoff scenario; training the model on data leading up to that point; and evaluating the model on data following it.

<img src="BlogImages/backtest.png"/>

In situations (like this) where we have multiple timeseries, we draw a distinction between:

* **target timeseries** (the things we want the model to forecast), and
* **[related timeseries](https://docs.aws.amazon.com/forecast/latest/dg/related-time-series-datasets.html)** (which we know ahead of time for our forecast window)

Therefore our train/test split will apply to the target timeseries only.

<img src="BlogImages/rts_viz.png">


### Choosing the Setup for our Bike Forecasting example

Our source data is at **hourly granularity** (sample frequency), and therefore this is the lower limit on what granularity we can sensibly forecast.

Whether we choose to use this granularity or aggregate up will guide what **forecast horizon** is appropriate:

* Model training will optimize overall accuracy for the given forecast horizon; so it should be set to the most useful horizon for the **business problem** (e.g. if I train a model to produce best-effort 1yr forecasts, but am actually only using it to plan 1month ahead in time, I may be missing out on performance)
* Because we'll be comparing recurrent neural models like DeepAR, we need to consider that some of these architectures work worse when the granularity is at least a certain fraction of the forecast window: The limit to what cycle-times of patterns can be learned are proportional to the sample frequency. Amazon Forecast in particular enforces a [limit](https://docs.aws.amazon.com/forecast/latest/dg/limits.html) of max 500 samples forecast horizon
* As discussed above, our algorithms will perform best when the data within our context (and horizon) window is approximately **stationary** and captures several cycles of the longest-period fluctuations - so quarterly or annual forecasts are probably not appropriate for the available data-set.

We'll train our predictors for a **14-day (2 week) horizon**, which:

* Roughly balances between the period over which half-decent weather forecasts can be made (in some countries at least); and the periods over which reasonable forward planning steps could be taken to deploy/move bikes to meet demand.
* Is short enough to let us keep our hourly sample and not have to mess around aggregating up

On the assumption (see overall volume graphs) that the end-of-year period is perhaps the *least* important time for the business, we'll further offset the final test cutoff back to 2012-12-01: Helping us avoid Christmas effects in our final evaluation window, but maximise the model's exposure to annual seasonality since only 2 years' data are provided.

In [ ]:
cutoff_test_start = "2012-12-01"

full_headers = full_df.columns.to_list()
timestamp_header = "timestamp"
target_headers = list(filter(lambda s: s.endswith(target_suffix), full_headers))
target_nontotal_headers = list(filter(lambda s: s != "total_demand", target_headers))
related_headers = list(filter(lambda s: (s not in target_headers) and (s != timestamp_header), full_headers))

In [ ]:
# Unpivoted dataframe of target variables, still sorted by timestamp:
target_full_df = full_df[[timestamp_header] + target_nontotal_headers].melt(
    id_vars=[timestamp_header],
    value_vars=target_nontotal_headers,
    var_name="customer_type",
    value_name="demand"
).sort_values(by=[timestamp_header, "customer_type"]).reset_index(drop=True)

# Strip "_demand" from the target IDs:
target_full_df["customer_type"] = target_full_df["customer_type"].apply(lambda s: s[0:-len(target_suffix)])

target_full_df.head()

In [ ]:
target_train_df = target_full_df[target_full_df[timestamp_header] < cutoff_test_start]
target_test_df = target_full_df[target_full_df[timestamp_header] >= cutoff_test_start]

related_df = full_df[[timestamp_header] + related_headers]

print(f"{len(target_train_df)} target training points")
print(f"{len(target_test_df)} target test points")
print(f"{len(related_df)} related timeseries points")

related_df.head()

In [ ]:
print("Writing dataframes to file...")
target_train_df.to_csv(
    f"./data/{target_train_filename}",
    index=False
)
target_test_df.to_csv(
    f"./data/{target_test_filename}",
    index=False
)
related_df.to_csv(
    f"./data/{related_filename}",
    index=False
)

print("Uploading dataframes to S3...")
s3.upload_file(
    Filename=f"./data/{target_train_filename}",
    Bucket=bucket,
    Key=f"{data_prefix}{target_train_filename}"
)
s3.upload_file(
    Filename=f"./data/{target_test_filename}",
    Bucket=bucket,
    Key=f"{data_prefix}{target_test_filename}"
)
s3.upload_file(
    Filename=f"./data/{related_filename}",
    Bucket=bucket,
    Key=f"{data_prefix}{related_filename}"
)
print("Done")


Now that our data is uploaded to S3, we can train models and compare their performance! Move on to the next notebook to start fitting predictors.